v1: 20, 10 and 15,10 gives 0.8377992451979948

v2: 50,30 and 15,30 gives 0.8136568467301302

v3: 50,30 and 15,10 gives 0.8041908409846222

v4: 100, 50 and 15,10 gives 0.7990184757505774

v5: 100, 50 only gives 0.86 -- chuck


In [52]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [53]:
songs_df=pd.read_csv('songs.csv')
test=pd.read_csv('test.csv')
df_submission = pd.read_csv('dummy_submission.csv')
df_new_train = pd.read_csv('train.csv')
train=df_new_train.reset_index(drop=True)

train, test = train_test_split(train, test_size=0.05, random_state=42, stratify = train['score'])

Y_test = test.score
test = test.drop(['score'], axis=1)

In [54]:
test.head()

,customer_id,song_id
17288,K24595,189
493127,L1416,467
155892,G37171,7033
15475,H18372,19
82767,K21545,9530


In [55]:
user_ids = train.customer_id.unique()
total_users = len(user_ids)

In [56]:
### TRUNCSVD ###

#Removing duplicates
song_matrix = pd.concat([train,test]).drop_duplicates(subset = ['customer_id','song_id'],keep = 'first')
#Creates a song matrix of #numofusers vs #noofsongs
song_matrix = song_matrix.pivot('customer_id','song_id','score')

song_means = song_matrix.mean()
user_means = song_matrix.mean(axis=1)
#Mean shifting
song_shifted_temp = song_matrix-song_means
song_shifted = song_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -song_shifted_temp.isnull()

def repeated_matrix_reconstruction(num_pcs,num_iterations):
    global song_shifted
    for i in range(num_iterations):
        SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
        SVD.fit(song_shifted)
        #For the ease of applying masks we work with pandas
        song_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(song_shifted)),columns=song_shifted.columns,index=song_shifted.index)
        loss = mean_squared_error(song_represented[mask].fillna(0),song_shifted_temp[mask].fillna(0))
        print('Iteration: {} , Loss: {} '.format(i,loss))
        #To just update the non-zero values of song_reprented values to the true ratings
        
        if i < (num_iterations - 1):
            song_represented[mask] = song_shifted_temp[mask]
        
        song_shifted = song_represented
            
    #Mean shifting it back
    song_mat = song_shifted + song_means
    song_mat = song_mat.clip(lower=1,upper=5)
    return song_mat
print("Starting truncated svd with number of components as 20")
representative_matrix_20 = repeated_matrix_reconstruction(50,30)
print("Done")
print("Starting truncated svd with number of components as 15")
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
print("Done")
#bagging
rating_matrix = (representative_matrix_15+representative_matrix_20)/2

Starting truncated svd with number of components as 20
Iteration: 0 , Loss: 0.0032432926872290914 
Iteration: 1 , Loss: 0.0029521382008729073 
Iteration: 2 , Loss: 0.002712757986518888 
Iteration: 3 , Loss: 0.0025124830255213604 
Iteration: 4 , Loss: 0.0023420139451461788 
Iteration: 5 , Loss: 0.002194457567270308 
Iteration: 6 , Loss: 0.00206485081139122 
Iteration: 7 , Loss: 0.00194960979826583 
Iteration: 8 , Loss: 0.0018460992328908962 
Iteration: 9 , Loss: 0.0017523397189909459 
Iteration: 10 , Loss: 0.0016668140380786802 
Iteration: 11 , Loss: 0.0015883372982482638 
Iteration: 12 , Loss: 0.0015159680213905923 
Iteration: 13 , Loss: 0.0014489460343280276 
Iteration: 14 , Loss: 0.0013866483425620962 
Iteration: 15 , Loss: 0.00132855731931162 
Iteration: 16 , Loss: 0.00127423746739771 
Iteration: 17 , Loss: 0.0012233182280310165 
Iteration: 18 , Loss: 0.001175481102978493 
Iteration: 19 , Loss: 0.0011304498851131273 
Iteration: 20 , Loss: 0.0010879831512197113 
Iteration: 21 , Loss:

In [59]:
rating_matrix = representative_matrix_20

In [60]:
trunc_prediction = np.zeros(len(test))
for i in range(len(test)):
    userid =  test.iloc[i,0]
    songid = test.iloc[i,1]
    trunc_prediction[i] = rating_matrix[rating_matrix.index==userid][songid].values[0]
    


In [61]:
PRED = trunc_prediction
PRED = np.around(PRED,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)


mean_squared_error(Y_test, PRED)

0.8690032670534558

In [58]:
PRED = trunc_prediction
PRED = np.around(PRED,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)


mean_squared_error(Y_test, PRED)

0.7990184757505774

In [62]:
songs_df=pd.read_csv('songs.csv')
test=pd.read_csv('test.csv')
df_submission = pd.read_csv('dummy_submission.csv')
df_new_train = pd.read_csv('train.csv')
train=df_new_train.reset_index(drop=True)

user_ids = train.customer_id.unique()
total_users = len(user_ids)


### TRUNCSVD ###

#Removing duplicates
song_matrix = pd.concat([train,test]).drop_duplicates(subset = ['customer_id','song_id'],keep = 'first')
#Creates a song matrix of #numofusers vs #noofsongs
song_matrix = song_matrix.pivot('customer_id','song_id','score')

song_means = song_matrix.mean()
user_means = song_matrix.mean(axis=1)
#Mean shifting
song_shifted_temp = song_matrix-song_means
song_shifted = song_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -song_shifted_temp.isnull()

def repeated_matrix_reconstruction(num_pcs,num_iterations):
    global song_shifted
    for i in range(num_iterations):
        SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
        SVD.fit(song_shifted)
        #For the ease of applying masks we work with pandas
        song_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(song_shifted)),columns=song_shifted.columns,index=song_shifted.index)
        loss = mean_squared_error(song_represented[mask].fillna(0),song_shifted_temp[mask].fillna(0))
        print('Iteration: {} , Loss: {} '.format(i,loss))
        #To just update the non-zero values of song_reprented values to the true ratings
        
        if i < (num_iterations - 1):
            song_represented[mask] = song_shifted_temp[mask]
        
        song_shifted = song_represented
            
    #Mean shifting it back
    song_mat = song_shifted + song_means
    song_mat = song_mat.clip(lower=1,upper=5)
    return song_mat
print("Starting truncated svd with number of components as 20")
representative_matrix_20 = repeated_matrix_reconstruction(50,30)
print("Done")
print("Starting truncated svd with number of components as 15")
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
print("Done")
#bagging
rating_matrix = (representative_matrix_15+representative_matrix_20)/2


trunc_prediction = np.zeros(len(test))
for i in range(len(test)):
    userid =  test.iloc[i,0]
    songid = test.iloc[i,1]
    trunc_prediction[i] = rating_matrix[rating_matrix.index==userid][songid].values[0]


PRED = trunc_prediction
PRED = np.around(PRED,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)


df_submission.score = PRED
df_submission.to_csv('./RM_v4.csv',index=False)


Starting truncated svd with number of components as 20
Iteration: 0 , Loss: 0.0037803236976831508 
Iteration: 1 , Loss: 0.003545879891388671 
Iteration: 2 , Loss: 0.0033485317373354874 
Iteration: 3 , Loss: 0.0031808493839277025 
Iteration: 4 , Loss: 0.003036367396052526 
Iteration: 5 , Loss: 0.002910004986944196 
Iteration: 6 , Loss: 0.0027980035709659207 
Iteration: 7 , Loss: 0.0026975717532641596 
Iteration: 8 , Loss: 0.002606611353641308 
Iteration: 9 , Loss: 0.002523525278231745 
Iteration: 10 , Loss: 0.002447081037011412 
Iteration: 11 , Loss: 0.002376313369831556 
Iteration: 12 , Loss: 0.002310454831821751 
Iteration: 13 , Loss: 0.002248886227144721 
Iteration: 14 , Loss: 0.002191101006688455 
Iteration: 15 , Loss: 0.002136679455856653 
Iteration: 16 , Loss: 0.0020852697617473 
Iteration: 17 , Loss: 0.002036573945384914 
Iteration: 18 , Loss: 0.00199033726567464 
Iteration: 19 , Loss: 0.0019463401274108583 
Iteration: 20 , Loss: 0.0019043918168481415 
Iteration: 21 , Loss: 0.001

In [11]:
df_submission.head()

,test_row_id,score
0,0,3.4
1,1,3.7
2,2,4.0
3,3,4.0
4,4,3.9
